In [1]:
import pandas as pd
df = pd.read_excel("C:/Users/SK-TECH/Downloads/household_new.xlsx")

In [2]:
df.drop(columns=["cet_cest_timestamp","area_offices","area_room_1","area_room_2","area_room_3","area_room_4","compressor","cooling_aggregate","cooling_pumps","dishwasher","ev","grid_import","pv_facade","pv_roof","refrigerator","ventilation"], inplace=True)
df.dropna(inplace=True)

In [6]:
df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'])
df.set_index("utc_timestamp", inplace=True)
df.sort_index(inplace=True)
df.reset_index(inplace=True)
df.head()
# .info(), .head(), .tail(), .describe()

,utc_timestamp,machine_1,machine_2,machine_3,machine_4,machine_5
0,2015-10-15 15:15:00+00:00,22.156,10.906,10.969,0.0,0.288
1,2015-10-15 15:30:00+00:00,29.953,13.375,14.062,0.0,0.378
2,2015-10-15 15:45:00+00:00,36.698,15.975,19.419,0.0,0.460
3,2015-10-15 16:00:00+00:00,42.782,18.614,23.502,0.0,0.552
4,2015-10-15 16:15:00+00:00,48.734,21.219,27.500,0.0,0.643


In [8]:
#count duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [15]:
df2 = df.melt(
    id_vars=["utc_timestamp"],
    value_vars= ["machine_1","machine_2","machine_3","machine_4","machine_5"],
    var_name="machine_col",
    value_name="power",

)
df2.head()

,utc_timestamp,machine_col,power
0,2015-10-15 15:15:00+00:00,machine_1,22.156
1,2015-10-15 15:30:00+00:00,machine_1,29.953
2,2015-10-15 15:45:00+00:00,machine_1,36.698
3,2015-10-15 16:00:00+00:00,machine_1,42.782
4,2015-10-15 16:15:00+00:00,machine_1,48.734


In [19]:
df2["machine_id"] = df2["machine_col"].str.replace("machine_", "").astype(int)
df2 = df2.sort_values(["machine_id", "utc_timestamp"])
print(df2.groupby("machine_id"))

In [32]:
df2["power_diff"] = df2.groupby("machine_id")["power"].diff()/0.25
df2["power_diff"] = df2["power_diff"].fillna(0)
df2.head()

,utc_timestamp,machine_col,power,machine_id,power_diff
0,2015-10-15 15:15:00+00:00,machine_1,22.156,1,0.0
1,2015-10-15 15:15:00+00:00,machine_3,10.969,3,0.0
2,2015-10-15 15:15:00+00:00,machine_4,0.000,4,0.0
3,2015-10-15 15:15:00+00:00,machine_2,10.906,2,0.0
4,2015-10-15 15:15:00+00:00,machine_5,0.288,5,0.0


In [31]:
df2 = df2.sort_values("utc_timestamp")
df2.head()

,utc_timestamp,machine_col,power,machine_id,power_diff
0,2015-10-15 15:15:00+00:00,machine_1,22.156,1,0.0
1,2015-10-15 15:15:00+00:00,machine_3,10.969,3,0.0
2,2015-10-15 15:15:00+00:00,machine_4,0.000,4,0.0
3,2015-10-15 15:15:00+00:00,machine_2,10.906,2,0.0
4,2015-10-15 15:15:00+00:00,machine_5,0.288,5,0.0


In [33]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
machine_onehot = encoder.fit_transform(df2[["machine_id"]])
machine_cols = [f"machine_{i}" for i in range(1, 6)]
df2[machine_cols] = machine_onehot

df2.drop(["machine_id","machine_col"], axis=1, inplace=True)
df2.head()

,utc_timestamp,power,power_diff,machine_1,machine_2,machine_3,machine_4,machine_5
0,2015-10-15 15:15:00+00:00,22.156,0.0,1.0,0.0,0.0,0.0,0.0
1,2015-10-15 15:15:00+00:00,10.969,0.0,0.0,0.0,1.0,0.0,0.0
2,2015-10-15 15:15:00+00:00,0.000,0.0,0.0,0.0,0.0,1.0,0.0
3,2015-10-15 15:15:00+00:00,10.906,0.0,0.0,1.0,0.0,0.0,0.0
4,2015-10-15 15:15:00+00:00,0.288,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
df2.to_csv("dataset_cleaned.csv", index=False)